In [1]:
import math
import matplotlib
from decimal import ROUND_HALF_UP, Decimal
import numpy as np
import pandas as pd
import seaborn as sns
import time
import talib
import copy

from datetime import date, datetime, time, timedelta
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import plot_model, timeseries_dataset_from_array

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#### Input params ##################
test_size = 0.2                 # proportion of dataset to be used as test set
val_size = 0.2                   # proportion of dataset to be used as cross-validation set
Nmax = 21                       # for feature at day t, we use lags from t-1, t-2, ..., t-N as features
                                # Nmax is the maximum N we are going to test

2023-05-31 15:56:22.696440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("~/study/optimal_investment/stock-market-prediction-project/data/train_files/stock_prices.csv")
# df = pd.read_csv("~/study/optimal_investment/stock-market-prediction-project/data/supplemental_files/stock_prices.csv")
data = df.copy()
data.drop(["RowId"], axis=1, inplace=True)
# print(df.shape)
data

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


### Calculate Adjusted Close value

In [3]:
def adjust_price(price):
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")
    def generate_adjusted_close(df):
        df = df.sort_values("Date", ascending=False)
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        df = df.sort_values("Date")
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    return price

In [4]:
df = adjust_price(df)
df = df.drop(["AdjustmentFactor", "ExpectedDividend", "SupervisionFlag", "CumulativeAdjustmentFactor", "RowId"], axis=1)
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target,AdjustedClose
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730,2742.0
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,0.002920,2738.0
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,-0.001092,2740.0
3,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,-0.005100,2748.0
4,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,-0.003295,2745.0


In [5]:
df["Close"] == df["AdjustedClose"]
df = df.drop(["AdjustedClose"], axis=1)
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,0.002920
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,-0.001092
3,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,-0.005100
4,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,-0.003295


### Calculate technical indicators

Let's see where is NaN values

In [7]:
df.isna().sum()

Date                 0
SecuritiesCode       0
Open              7608
High              7608
Low               7608
Close             7608
Volume               0
Target             238
dtype: int64

In [8]:
high_nans = df[pd.isnull(df.High)]
h = high_nans[["SecuritiesCode", "High"]].set_index("SecuritiesCode")
h.fillna(0, inplace=True)
print(f"Unique Nan count for each stock: {h.groupby(level=0).count().High.unique()}")
# h.groupby(level=0).count()
high_nans[df.SecuritiesCode == 1787]

Unique Nan count for each stock: [  1   3  18   5 280  68  20 283   4   2  21  12  14  17  26   7 347   8
  83  97  82  45 237 115   9   6 282 225 285  16 269  27  89 279 186  52
  23  55  76  19 106  22 101  62  77  37  13 105 163 144 161  35 351  24
  36  46 174]


,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target
36914,2017-01-17,1787,NaN,NaN,NaN,NaN,0,-0.004049
36917,2017-01-20,1787,NaN,NaN,NaN,NaN,0,0.000000
36919,2017-01-24,1787,NaN,NaN,NaN,NaN,0,0.008000
36926,2017-02-02,1787,NaN,NaN,NaN,NaN,0,0.000000
36929,2017-02-07,1787,NaN,NaN,NaN,NaN,0,0.003197
...,...,...,...,...,...,...,...,...
38096,2021-11-17,1787,NaN,NaN,NaN,NaN,0,0.000000
38097,2021-11-18,1787,NaN,NaN,NaN,NaN,0,0.014235
38098,2021-11-19,1787,NaN,NaN,NaN,NaN,0,0.008772
38104,2021-11-30,1787,NaN,NaN,NaN,NaN,0,0.031301


It turns out that for each time series matching a particular stock, we have NaN values twice or more in a row

Fill NaN values using splain interpolation:

In [9]:
sec_codes = df.SecuritiesCode.unique().tolist()
for code in sec_codes:
    df.loc[df.SecuritiesCode == code, "Target"] = df[df.SecuritiesCode == code]["Target"].interpolate(method='spline', order=3, limit_direction='both')
    df.loc[df.SecuritiesCode == code, "High"] = df[df.SecuritiesCode == code]["High"].interpolate(method='spline', order=3, limit_direction='both')
    df.loc[df.SecuritiesCode == code, "Low"] = df[df.SecuritiesCode == code]["Low"].interpolate(method='spline', order=3, limit_direction='both')
    df.loc[df.SecuritiesCode == code , "Close"] = df[df.SecuritiesCode == code]["Close"].interpolate(method='spline', order=3, limit_direction='both')
    df.loc[df.SecuritiesCode == code , "Open"] = df[df.SecuritiesCode == code]["Open"].interpolate(method='spline', order=3, limit_direction='both')

In [10]:
df.isna().sum()

Date              0
SecuritiesCode    0
Open              0
High              0
Low               0
Close             0
Volume            0
Target            0
dtype: int64

In [11]:
# initialize emtpy DataFrame with technical indicators
ti = ["SMA", "WMA", "MOM", "ADO", "CCI", "RSI", "LWR", "MACD", "STCK", "STCD"]
df_ti = pd.DataFrame(np.nan, index=[i for i in range(df.shape[0])], columns=ti)
df_ti

,SMA,WMA,MOM,ADO,CCI,RSI,LWR,MACD,STCK,STCD
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2332526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2332527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2332528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2332529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
SMA_index = df_ti.columns.get_loc("SMA")
WMA_index = df_ti.columns.get_loc("WMA")
MOM_index = df_ti.columns.get_loc("MOM")
ADO_index = df_ti.columns.get_loc("ADO")
CCI_index = df_ti.columns.get_loc("CCI")
RSI_index = df_ti.columns.get_loc("RSI")
LWR_index = df_ti.columns.get_loc("LWR")
MACD_index = df_ti.columns.get_loc("MACD")
STCK_index = df_ti.columns.get_loc("STCK")
STCD_index = df_ti.columns.get_loc("STCD")

for code in sec_codes:
    stock_indexes = df[df.SecuritiesCode == code].index.to_list()
    stock_values = df.iloc[stock_indexes]
    df_ti.iloc[stock_indexes, SMA_index] = talib.SMA(stock_values.Close, timeperiod=10)
    df_ti.iloc[stock_indexes, WMA_index] = talib.WMA(stock_values.Close, timeperiod=14)
    df_ti.iloc[stock_indexes, MOM_index] = talib.MOM(stock_values.Close, timeperiod=10)
    df_ti.iloc[stock_indexes, ADO_index] = talib.ADOSC(stock_values.High, stock_values.Low, stock_values.Close, stock_values.Volume)
    df_ti.iloc[stock_indexes, CCI_index] = talib.CCI(stock_values.High, stock_values.Low, stock_values.Close, timeperiod=10)
    df_ti.iloc[stock_indexes, RSI_index] = talib.RSI(stock_values.Close, timeperiod=10)
    df_ti.iloc[stock_indexes, LWR_index] = talib.WILLR(stock_values.High, stock_values.Low, stock_values.Close, timeperiod=10)
    df_ti.iloc[stock_indexes, MACD_index] = talib.MACD(stock_values.Close, signalperiod=10)[0]
    df_ti.iloc[stock_indexes, STCK_index] = talib.STOCH(stock_values.High, stock_values.Low, stock_values.Close)[0]
    df_ti.iloc[stock_indexes, STCD_index] = talib.STOCH(stock_values.High, stock_values.Low, stock_values.Close)[1]    

In [13]:
# df_ti.SMA
df_ti[:40]

,SMA,WMA,MOM,ADO,CCI,RSI,LWR,MACD,STCK,STCD
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.920764,36.255299
9,2725.0,NaN,NaN,2936.575048,-129.141566,NaN,-82.432432,NaN,13.068120,22.435180


In [14]:
df_res = pd.concat([df, df_ti], axis=1)
df_res[:40]
# type(df), type(df_ti)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target,SMA,WMA,MOM,ADO,CCI,RSI,LWR,MACD,STCK,STCD
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,0.002920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,-0.001092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,-0.005100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,-0.003295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017-01-12,1301,2745.0,2747.0,2703.0,2731.0,28700,-0.006613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017-01-13,1301,2707.0,2730.0,2707.0,2722.0,19400,-0.006657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017-01-16,1301,2725.0,2725.0,2696.0,2704.0,20100,0.002978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-01-17,1301,2702.0,2704.0,2682.0,2686.0,18400,0.001856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.920764,36.255299
9,2017-01-18,1301,2689.0,2695.0,2681.0,2694.0,12100,0.014079,2725.0,NaN,NaN,2936.575048,-129.141566,NaN,-82.432432,NaN,13.068120,22.435180


Remove first 34 rows with NaN values for each stock:

In [15]:
for code in sec_codes:
    stock_indexes_to_remove = df[df.SecuritiesCode == code].index.to_list()[0:34]
    df_res.drop(stock_indexes_to_remove, axis=0, inplace=True)
df_res.shape

(2264531, 18)

In [16]:
df_res.isna().sum()

Date              0
SecuritiesCode    0
Open              0
High              0
Low               0
Close             0
Volume            0
Target            0
SMA               0
WMA               0
MOM               0
ADO               0
CCI               0
RSI               0
LWR               0
MACD              0
STCK              0
STCD              0
dtype: int64

In [17]:
df_res.to_csv("~/study/optimal_investment/stock-market-prediction-project/data/my/test_data_2.csv", index=False)